In [2]:
import pandas as pd

In [3]:
from nltk import word_tokenize
MAX_LEN = 13

def get3DMatrix(captions, embeddings):
    res = []
    for caption in captions:
        sentence = [np.asarray([0]*300,dtype='float32')]*MAX_LEN
        words = word_tokenize(caption)
        for i, word in enumerate(words):
            sentence[i] = embeddings.get(word, np.asarray([0]*300,dtype='float32'))
        res.append(sentence)
    return np.asarray(res)

matrix_map = {}
with open('train.csv') as f:
    for i, line in enumerate(f):
        print(i, end='\r')
        image_id, captions, questions, answers = line.split('::zz')
        captions = captions.split('#:#')
        matrix_map[image_id] = get3DMatrix(captions, embeddings)

In [4]:
data = {'image_id': [], 'captions': [], 'questions': [], 'answers': [], 'question_ids': [], 'caption_matrix': []}
with open('train.csv') as f:
    for i, line in enumerate(f):
        print(i, end='\r')
        image_id, captions, questions, answers = line.split('::zz')
        captions = captions.split('#:#')
        questions = questions.split('#:#')
        answers = answers.split('#:#')
        question_answers = []
        questionsMod = {}
        answersMod = {}
        for i in range(len(questions)):
            if i%2 == 0:
                questionsMod[questions[i]] = questions[i+1]
        for i in range((len(answers))):
            if i %2 == 0:
                answersMod[answers[i]] = answers[i+1]
        for question_id in questionsMod:
            question_answers.append((question_id, questionsMod[question_id], answersMod[question_id]))
        for question_id, question, answer in question_answers:
            data['image_id'].append(image_id)
            data['captions'].append(captions)
            data['questions'].append(question)
            data['answers'].append(answer)
            data['question_ids'].append(question_id)
            data['caption_matrix'].append(matrix_map[image_id])

In [5]:
df = pd.DataFrame(data=data)
print(df.head())
df.to_pickle('trainv1.pkl')

         answers                                     caption_matrix  \
0    living room  [[[0.68758, -0.34531, -0.18579, 0.2387, -0.071...   
1            yes  [[[0.68758, -0.34531, -0.18579, 0.2387, -0.071...   
2  living room\n  [[[0.68758, -0.34531, -0.18579, 0.2387, -0.071...   
3            yes  [[[0.21497, -0.29447, -0.47612, -0.029865, 0.2...   
4              1  [[[0.21497, -0.29447, -0.47612, -0.029865, 0.2...   

                                            captions image_id question_ids  \
0  [window with white frame, black leather couch,...    98304     98304001   
1  [window with white frame, black leather couch,...    98304     98304000   
2  [window with white frame, black leather couch,...    98304     98304002   
3  [train on the tracks, train on the tracks, yel...    32773     32773001   
4  [train on the tracks, train on the tracks, yel...    32773     32773000   

                                questions  
0             Where in the house is this?  
1     Is the fur

In [1]:
import numpy as np
embeddings = {}
cnt = 0
with open('./../embeddings/glove.840B.300d.txt','r') as f:
    for i, line in enumerate(f):
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:],dtype='float32')
        except Exception as ex:
            cnt += 1
            print("ex:{}/i:{}".format(cnt, i))
            continue
        embeddings[word] = coefs

ex:1/i:52343
ex:2/i:128261
ex:3/i:151102
ex:4/i:200668
ex:5/i:209833
ex:6/i:220779
ex:7/i:253461
ex:8/i:365745
ex:9/i:532048
ex:10/i:717302
ex:11/i:994818
ex:12/i:1123331
ex:13/i:1148409
ex:14/i:1352110
ex:15/i:1499727
ex:16/i:1533809


In [72]:
df = pd.read_pickle('train.pkl')

76380

In [21]:
mx = 0
for index, row in df.iterrows():
    captions = row['captions']
    for caption in captions:
        mx = max(len(caption.split(' ')), mx)
print(mx)

11


In [73]:
from nltk import word_tokenize
MAX_LEN = 11

def get3DMatrix(captions, embeddings):
    res = []
    for caption in captions:
        sentence = [np.asarray([0]*300,dtype='float32')]*MAX_LEN
        words = word_tokenize(caption)
        for i, word in enumerate(words):
            sentence[i] = embeddings.get(word, np.asarray([0]*300,dtype='float32'))
        res.append(sentence)
    return np.asarray(res)
        
mat = get3DMatrix(captions, embeddings)
print(mat.shape)

(10, 11, 300)


In [1]:
import json
with open('results_val/results_val.json') as f:
    cont = f.read()
results = json.loads(cont)
with open('results_val/questions.json') as f:
    cont = f.read()
questions = json.loads(cont)
with open('results_val/annotations.json') as f:
    cont = f.read()
answers = json.loads(cont)

In [13]:
import numpy as np
def euclideanDist(p1, p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
boxes = results['results'][0]['boxes'][:10]
points = [((box[0]+box[2])/2, (box[1]+box[2])/2) for box in boxes]
res = np.asarray([np.asarray([0]*10) for _ in range(10)])
for i in range(10):
    for j in range(10):
        res[i][j] = euclideanDist(points[i], points[j])
for row in res:
    print(row)

[  0  57 258 102  94 166 166 141 184 174]
[ 57   0 201 157 134 111 162 198 208 138]
[258 201   0 353 313  95 257 399 369 161]
[102 157 353   0  62 259 184  57 216 238]
[ 94 134 313  62   0 218 122 119 255 183]
[166 111  95 259 218   0 177 306 301  91]
[166 162 257 184 122 177   0 242 350  97]
[141 198 399  57 119 306 242   0 197 293]
[184 208 369 216 255 301 350 197   0 346]
[174 138 161 238 183  91  97 293 346   0]


In [42]:
mp = {}
for result in results['results'][:6750]:
    img_id = int(result['img_name'].split('.')[0].split('_')[-1])
    mp[img_id] = []
    mp[img_id].append(result['captions'][:10])
    mp[img_id].append([])
    mp[img_id].append([])
    mp[img_id].append(result['boxes'][:10])
for question in questions['questions']:
    img_id, question, question_id = question['image_id'], question['question'], question['question_id']
    if img_id in mp:
        mp[img_id][1].append([question_id, question])
for answer in answers['annotations']:
    img_id, multiple_choice_answer, question_id = answer['image_id'], answer['multiple_choice_answer'], answer['question_id']
    if img_id in mp:
        mp[img_id][2].append([question_id, multiple_choice_answer])

In [36]:
df = pd.read_pickle('val.pkl')
mx = 0
for index, row in df.iterrows():
    captions = row['captions']
    for caption in captions:
        mx = max(len(caption.split(' ')), mx)
print(mx)

13


In [37]:
df.size

607560

In [45]:
from nltk import word_tokenize
MAX_LEN = 13

def get3DMatrix(captions, embeddings):
    res = []
    for caption in captions:
        sentence = [np.asarray([0]*300,dtype='float32')]*MAX_LEN
        words = word_tokenize(caption)
        for i, word in enumerate(words):
            sentence[i] = embeddings.get(word, np.asarray([0]*300,dtype='float32'))
        res.append(sentence)
    return np.asarray(res)

matrix_map = {}
for image_id in mp:
    captions = mp[image_id][0]
    matrix_map[image_id] = get3DMatrix(captions, embeddings)

In [46]:
data = {'image_id': [], 'captions': [], 'questions': [], 'answers': [], 'question_ids': [], 'caption_matrix': []}
for image_id in mp:
    captions, questions, answers = mp[image_id]
    question_answers = []
    questionsMod = {}
    answersMod = {}
    for q_id, q in questions:
        questionsMod[q_id] = q
    for q_id, a in answers:
        answersMod[q_id] = a
    for question_id in questionsMod:
        question_answers.append((question_id, questionsMod[question_id], answersMod[question_id]))
    for question_id, question, answer in question_answers:
        data['image_id'].append(image_id)
        data['captions'].append(captions)
        data['questions'].append(question)
        data['answers'].append(answer)
        data['question_ids'].append(question_id)
        data['caption_matrix'].append(matrix_map[image_id])

In [47]:
import pandas as pd
df = pd.DataFrame(data=data)
print(df.head())
df.to_pickle('valv1.pkl')

       answers                                     caption_matrix  \
0           no  [[[0.043798, 0.024779, -0.20937, 0.49745, 0.36...   
1  city street  [[[0.043798, 0.024779, -0.20937, 0.49745, 0.36...   
2          yes  [[[0.043798, 0.024779, -0.20937, 0.49745, 0.36...   
3        black  [[[0.043798, 0.024779, -0.20937, 0.49745, 0.36...   
4        brush  [[[0.043798, 0.024779, -0.20937, 0.49745, 0.36...   

                                            captions  image_id  question_ids  \
0  [a traffic light, a red traffic light, a city ...    360449       3604490   
1  [a traffic light, a red traffic light, a city ...    360449       3604491   
2  [a traffic light, a red traffic light, a city ...    360449       3604492   
3  [a cat in a sink, the cat is looking at the ca...    245764       2457640   
4  [a cat in a sink, the cat is looking at the ca...    245764       2457641   

                                           questions  
0  Is this the best sunset picture you've ever se

In [1]:
word_idx = {}
with open('./../embeddings/word_idx') as f:
    for line in f:
        word, idx = line.split(' ')
        word_idx[word] = idx

In [4]:
import pickle
with open('../embeddings/word_idx_1', 'wb') as f:
    pickle.dump(word_idx, f)

In [7]:
path = '../embeddings/word_idx_1'
with open(path,'rb') as file:
    word_idx = pickle.load(file)

In [1]:
import pandas as pd
df = pd.read_pickle('../test.pkl')

In [3]:
df.head(15)

,answers,captions,image_id,question_ids,questions,top_answers
0,[no],"[[a traffic light, a red traffic light, a city...",[360449],[3604490],Is this the best sunset picture you've ever seen?,"[no, yes, yes\n]"
1,[city street],"[[a traffic light, a red traffic light, a city...",[360449],[3604491],What street is this?,"[racing, beige, beach]"
2,[yes],"[[a traffic light, a red traffic light, a city...",[360449],[3604492],Are there lots of lights in this street?,"[no, yes, yes\n]"
3,[black],"[[a cat in a sink, the cat is looking at the c...",[245764],[2457640],What color is the cat sitting on top of the to...,"[blue, green, white]"
4,[brush],"[[a cat in a sink, the cat is looking at the c...",[245764],[2457641],What is next to the toilet paper?,"[soup, water, phone]"
5,[toilet],"[[a cat in a sink, the cat is looking at the c...",[245764],[2457642],Where is the cat?,"[parking lot, plane, umbrella]"
6,[yes],"[[man carrying a surfboard, two people walking...",[434177],[4341770],Will the people's feet get wet?,"[no, yes, no\n]"
7,[surfboards],"[[man carrying a surfboard, two people walking...",[434177],[4341771],What are the people caring in their arms?,"[cake, down, 9]"
8,[surfboards],"[[man carrying a surfboard, two people walking...",[434177],[4341772],What are they carrying?,"[taxi, cake\n, flying]"
9,[no],"[[a red fire hydrant, red fire hydrant, red fi...",[229385],[2293850],Does the hydrant need repainting?,"[yes\n, no, yes]"
